In [ ]:
! pip install diffusers transformers accelerate --upgrade

In [ ]:
from diffusers import AutoPipelineForText2Image
from google.colab import drive
from os import path

import random
import torch
import numpy as np

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Set random seeds to 42 for reproducibility
np.random.seed(42)
torch.manual_seed(42)
random.seed(42)

In [ ]:
PROMPT_PATH = f'/content/drive/My Drive/CLIP_Facial_Impressions/stimuli/attributes.txt'
OPPOSITE_PATH = f'/content/drive/My Drive/CLIP_Facial_Impressions/stimuli/attributes_opposites.txt'
SAVE_PATH  = f'/content/drive/My Drive/CLIP_Facial_Impressions/sdxl_images'

In [ ]:
prompt_attributes = {line.split(':')[0]:line.split(':')[-1].replace('a photo of','a realistic portrait photo of') for line in open(PROMPT_PATH).read().split('\n') if line}
opposite_prompts = {line.split(':')[0]:line.split(':')[-1].replace('a photo of','a realistic portrait photo of') for line in open(OPPOSITE_PATH).read().split('\n') if line}


In [ ]:
pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")

In [ ]:
seeds = [random.randint(0,100000) for _ in range(100)]
seedstr = '\n'.join([str(i) for i in seeds])
with open(f'/content/drive/My Drive/CLIP_Facial_Impressions/sdxl_seeds.txt', 'w') as f:
  f.write(seedstr)

In [ ]:
for attribute in list(prompt_attributes.keys())[-3:]:
  print(attribute)

  positive_prompt = prompt_attributes[attribute]
  negative_prompt = opposite_prompts[attribute]

  for i in range(25):
    image = pipe(prompt=positive_prompt, num_inference_steps=4, guidance_scale=0.0, seed=seeds[i]).images[0]
    image.save(path.join(SAVE_PATH,f'{attribute}_positive_{i}.png'))

  for i in range(25):
    image = pipe(prompt=negative_prompt, num_inference_steps=4, guidance_scale=0.0, seed=seeds[i]).images[0]
    image.save(path.join(SAVE_PATH,f'{attribute}_negative_{i}.png'))